In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pickle
import re

In [2]:
with open('../capstone_data/rhc_qcor_info.pickle', 'rb') as file:
     rhc_qcor = pickle.load(file)
        
with open('../capstone_data/fqhc_qcor_info.pickle', 'rb') as file:
     fqhc_qcor = pickle.load(file)
        
with open('../capstone_data/cah_qcor_info.pickle', 'rb') as file:
     cah_qcor = pickle.load(file)               

In [3]:
rhc_hrsa = pd.read_excel('../capstone_data/HRSA_RHC_details.xlsx')

fqhc_hrsa = pd.read_excel('../capstone_data/HRSA_FQHC_details.xlsx')

cah_hrsa = pd.read_excel('../capstone_data/HRSA_CAH_details.xlsx')

In [4]:
#pulling in information that links zip codes to counties, confirming no zipcodes cross two or more counties
zips_counties =  pd.read_csv('../capstone_data/zip_code_database.csv')
tn_zips_counties= zips_counties.loc[zips_counties['state'] == 'TN']
tn_zips_counties[['zip', 'county']].value_counts()

zip    county           
37010  Montgomery County    1
38057  Fayette County       1
38059  Dyer County          1
38060  Fayette County       1
38061  Hardeman County      1
                           ..
37403  Hamilton County      1
37404  Hamilton County      1
37405  Hamilton County      1
37406  Hamilton County      1
38589  Overton County       1
Length: 795, dtype: int64

In [5]:
zc_dict = tn_zips_counties.set_index('zip')['county'].to_dict()
zc_dict

{37010: 'Montgomery County',
 37011: 'Davidson County',
 37012: 'DeKalb County',
 37013: 'Davidson County',
 37014: 'Williamson County',
 37015: 'Cheatham County',
 37016: 'Cannon County',
 37018: 'Coffee County',
 37019: 'Marshall County',
 37020: 'Bedford County',
 37022: 'Sumner County',
 37023: 'Stewart County',
 37024: 'Williamson County',
 37025: 'Hickman County',
 37026: 'Cannon County',
 37027: 'Williamson County',
 37028: 'Stewart County',
 37029: 'Dickson County',
 37030: 'Smith County',
 37031: 'Sumner County',
 37032: 'Robertson County',
 37033: 'Hickman County',
 37034: 'Marshall County',
 37035: 'Cheatham County',
 37036: 'Dickson County',
 37037: 'Rutherford County',
 37040: 'Montgomery County',
 37041: 'Montgomery County',
 37042: 'Montgomery County',
 37043: 'Montgomery County',
 37044: 'Montgomery County',
 37046: 'Williamson County',
 37047: 'Marshall County',
 37048: 'Sumner County',
 37049: 'Robertson County',
 37050: 'Stewart County',
 37051: 'Dickson County',
 37

In [7]:
tn_rural_counties =  pd.read_csv('../capstone_data/tn_county_pop_rural.csv')
county_rural_dict = tn_rural_counties.set_index('County')['HRSA Rural Status'].to_dict()
county_rural_dict

{'Anderson County': 'Non-Rural',
 'Bedford County': 'Rural',
 'Benton County': 'Rural',
 'Bledsoe County': 'Rural',
 'Blount County': 'Partially Rural',
 'Bradley County': 'Non-Rural',
 'Campbell County': 'Rural',
 'Cannon County': 'Rural',
 'Carroll County': 'Rural',
 'Carter County': 'Non-Rural',
 'Cheatham County': 'Rural',
 'Chester County': 'Rural',
 'Claiborne County': 'Rural',
 'Clay County': 'Rural',
 'Cocke County': 'Rural',
 'Coffee County': 'Rural',
 'Crockett County': 'Rural',
 'Cumberland County': 'Rural',
 'Davidson County': 'Non-Rural',
 'Decatur County': 'Rural',
 'DeKalb County': 'Rural',
 'Dickson County': 'Rural',
 'Dyer County': 'Rural',
 'Fayette County': 'Non-Rural',
 'Fentress County': 'Rural',
 'Franklin County': 'Rural',
 'Gibson County': 'Rural',
 'Giles County': 'Rural',
 'Grainger County': 'Rural',
 'Greene County': 'Rural',
 'Grundy County': 'Rural',
 'Hamblen County': 'Non-Rural',
 'Hamilton County': 'Non-Rural',
 'Hancock County': 'Rural',
 'Hardeman Coun

In [8]:
rhc_qcor['Origin'] = 'Qcor'
rhc_hrsa['Origin'] = 'HRSA'

rhc_hrsa['Provider #'] = rhc_hrsa['Provider #'].astype(str)

rhc = pd.merge(rhc_qcor, rhc_hrsa, how='outer', left_on = ['cert_num'], 
               right_on = ['Provider #'])

rhc['facility_name'] = rhc['facility_name'].fillna(rhc['Site Name'])
rhc['Telephone #'] = rhc['Telephone #'].fillna(rhc['phone'])
rhc['cert_num'] = rhc['cert_num'].fillna(rhc['Provider #'])
rhc['facility_type'] = rhc['facility_type'].fillna(rhc['Site Subcategory'])
rhc['Address'] = rhc['Address'].fillna(rhc['address'])
rhc['State'] = rhc['State'].fillna('TN')

for ind, row in rhc.iterrows():
    if pd.isna(rhc['City'].iloc[ind]) == True:
        rhc.loc[ind, 'City'] = re.search('(.+),\s', row['city_state']).group(1)
        
for ind, row in rhc.iterrows():
    if pd.isna(rhc['ZIP'].iloc[ind]) == True:
        rhc.loc[ind, 'ZIP'] = re.search('(\d{5})', row['city_state']).group(1)
        
rhc = rhc.drop(columns = ['Site Name', 'phone', 'Provider #', 'Site Category', 'Site Subcategory', 
                          'address', 'city_state', 'HHS Region', 'State County FIPS Code', 'Type',
                         '# Direct Care Prsnl. FTEs', 'Total FTEs'])

rhc['facility_name'] = rhc['facility_name'].str.title()
rhc['City'] = rhc['City'].str.title()
rhc['Address'] = rhc['Address'].str.title()

rhc['accreditation_organization'] = 'Not Applicable' 
rhc['accreditation_type'] = 'Not Applicable'

rhc['zip_5'] = rhc['ZIP'].str.extract('(\d{5})')
rhc['zip_5'] = rhc['zip_5'].astype(int)      
rhc['County'] = rhc['County'].fillna(rhc['zip_5'].map(zc_dict))
rhc = rhc.drop(columns = ['zip_5'])

rhc['Rural Status'] = rhc['County'].map(county_rural_dict)

rhc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 244 entries, 0 to 243
Data columns (total 33 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   facility_name                           244 non-null    object 
 1   cert_num                                244 non-null    object 
 2   facility_type                           244 non-null    object 
 3   participation_date                      243 non-null    object 
 4   ownership_type                          243 non-null    object 
 5   Origin_x                                243 non-null    object 
 6   County                                  244 non-null    object 
 7   State                                   244 non-null    object 
 8   Address                                 244 non-null    object 
 9   City                                    244 non-null    object 
 10  ZIP                                     244 non-null    object

In [9]:
fqhc_qcor['Origin'] = 'Qcor'
fqhc_hrsa['Origin'] = 'HRSA'

fqhc_hrsa['Provider #'] = fqhc_hrsa['Provider #'].astype(str)

fqhc = pd.merge(fqhc_qcor, fqhc_hrsa, how='outer', left_on = ['cert_num'], 
               right_on = ['Provider #'])

fqhc['facility_name'] = fqhc['facility_name'].fillna(fqhc['Site Name'])
fqhc['Telephone #'] = fqhc['Telephone #'].fillna(fqhc['phone'])
fqhc['cert_num'] = fqhc['cert_num'].fillna(fqhc['Provider #'])
fqhc['facility_type'] = fqhc['facility_type'].fillna(fqhc['Site Subcategory'])
fqhc['Address'] = fqhc['Address'].fillna(fqhc['address'])
fqhc['State'] = fqhc['State'].fillna('TN')


for ind, row in fqhc.iterrows():
    if pd.isna(fqhc['City'].iloc[ind]) == True:
        fqhc.loc[ind, 'City'] = re.search('(.+),\s', row['city_state']).group(1)
        
for ind, row in fqhc.iterrows():
    if pd.isna(fqhc['ZIP'].iloc[ind]) == True:
        fqhc.loc[ind, 'ZIP'] = re.search('(\d{5})', row['city_state']).group(1)

fqhc = fqhc.drop(columns = ['Site Name', 'phone', 'Provider #', 'Site Category', 'Site Subcategory', 
                          'address', 'city_state', 'HHS Region', 'State County FIPS Code', 'Type',
                           '# Direct Care Prsnl. FTEs', 'Total FTEs'])

fqhc['facility_name'] = fqhc['facility_name'].str.title()
fqhc['City'] = fqhc['City'].str.title()
fqhc['Address'] = fqhc['Address'].str.title()

fqhc['Rural Status'] = fqhc['County'].map(county_rural_dict)

fqhc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 166 entries, 0 to 165
Data columns (total 31 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   facility_name                           166 non-null    object 
 1   cert_num                                166 non-null    object 
 2   facility_type                           166 non-null    object 
 3   participation_date                      166 non-null    object 
 4   ownership_type                          166 non-null    object 
 5   Origin_x                                166 non-null    object 
 6   County                                  166 non-null    object 
 7   State                                   166 non-null    object 
 8   Address                                 166 non-null    object 
 9   City                                    166 non-null    object 
 10  ZIP                                     166 non-null    object

In [10]:
#fqhc.to_csv('../capstone_data/fqhc_tn.csv')

In [11]:
cah_hrsa['Provider #'] = cah_hrsa['Provider #'].astype(str)

cah = pd.merge(cah_qcor, cah_hrsa, how='outer', left_on = ['cert_num'], 
               right_on = ['Provider #'])

cah['facility_name'] = cah['facility_name'].fillna(cah['Site Name'])
cah['Telephone #'] = cah['Telephone #'].fillna(cah['phone'])
cah['cert_num'] = cah['cert_num'].fillna(cah['Provider #'])
cah['Address'] = cah['Address'].fillna(cah['address'])
cah['State'] = cah['State'].fillna('TN')


for ind, row in cah.iterrows():
    if pd.isna(cah['City'].iloc[ind]) == True:
        cah.loc[ind, 'City'] = re.search('(.+),\s', row['city_state']).group(1)
        
for ind, row in cah.iterrows():
    if pd.isna(cah['ZIP'].iloc[ind]) == True:
        cah.loc[ind, 'ZIP'] = re.search('(\d{5})', row['city_state']).group(1)

cah['facility_type'] = cah['Site Subcategory']
cah['facility_name'] = cah['facility_name'].str.title()
cah['accreditation_organization'] = cah['accreditation_organization'].str.title()
cah['accreditation_organization'] = cah['accreditation_organization'].str.replace('Dnv', 'DNV')

cah = cah.drop(columns = ['Site Name', 'Site Subcategory', 'facility_subtype','phone', 'Provider #', 
                          'Site Category', 'address', 'city_state', 'HHS Region', 'State County FIPS Code', 
                          'Type', '# Direct Care Prsnl. FTEs', 'Total FTEs'])

cah['Rural Status'] = cah['County'].map(county_rural_dict)

cah.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16 entries, 0 to 15
Data columns (total 32 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   facility_name                           16 non-null     object 
 1   cert_num                                16 non-null     object 
 2   facility_type                           16 non-null     object 
 3   participation_date                      16 non-null     object 
 4   ownership_type                          16 non-null     object 
 5   accreditation_organization              16 non-null     object 
 6   accreditation_type                      16 non-null     object 
 7   number_of_beds                          16 non-null     object 
 8   County                                  16 non-null     object 
 9   State                                   16 non-null     object 
 10  Address                                 16 non-null     object 


In [12]:
#cah.to_csv('../capstone_data/cah_tn.csv')

In [13]:
rhc.loc[rhc['Latitude'].isna()]

,facility_name,cert_num,facility_type,participation_date,ownership_type,Origin_x,County,State,Address,City,...,Is Federally Funded Health Center Site,Is Hospital-Based Site,Is Rural Health Clinic Site,Approved RHC Provider #,Rural Status,Longitude,Latitude,Origin_y,accreditation_organization,accreditation_type
54,Faith Family Wellness Clinic Pllc,883909,Rural Health Clinic,04/28/2022,For Profit Individual,Qcor,Bedford County,TN,2012 N Main Street,Shelbyville,...,NaN,NaN,NaN,NaN,Rural,NaN,NaN,NaN,Not Applicable,Not Applicable
57,Family Care Walk In Clinic Inc,883910,Rural Health Clinic,05/18/2022,For Profit Corporation,Qcor,Gibson County,TN,15001 S 1St Street,Milan,...,NaN,NaN,NaN,NaN,Rural,NaN,NaN,NaN,Not Applicable,Not Applicable
72,Fast Pace Medical Clinic Pllc - Athens,883900,Rural Health Clinic,01/21/2022,For Profit Individual,Qcor,McMinn County,TN,1019 Decatur Pike,Athens,...,NaN,NaN,NaN,NaN,Rural,NaN,NaN,NaN,Not Applicable,Not Applicable
76,Fast Pace Medical Clinic Pllc - Carthage,883901,Rural Health Clinic,02/01/2022,For Profit Individual,Qcor,Smith County,TN,573 Carthage Bypass,Carthage,...,NaN,NaN,NaN,NaN,Rural,NaN,NaN,NaN,Not Applicable,Not Applicable
81,Fast Pace Medical Clinic Pllc - Dunlap,883902,Rural Health Clinic,01/21/2022,For Profit Individual,Qcor,Sequatchie County,TN,7449 State Route 28,Dunlap,...,NaN,NaN,NaN,NaN,Non-Rural,NaN,NaN,NaN,Not Applicable,Not Applicable
101,Fast Pace Medical Clinic Pllc - Rockwood,883903,Rural Health Clinic,01/21/2022,For Profit Individual,Qcor,Roane County,TN,1118 N Gateway Ave,Rockwood,...,NaN,NaN,NaN,NaN,Non-Rural,NaN,NaN,NaN,Not Applicable,Not Applicable
102,Fast Pace Medical Clinic Pllc - Rocky Top,883904,Rural Health Clinic,02/01/2022,For Profit Individual,Qcor,Anderson County,TN,5100 N Main Street,Rocky Top,...,NaN,NaN,NaN,NaN,Non-Rural,NaN,NaN,NaN,Not Applicable,Not Applicable
105,Fast Pace Medical Clinic Pllc -Smithville,883908,Rural Health Clinic,02/01/2022,For Profit Individual,Qcor,DeKalb County,TN,101 W Broad St,Smithville,...,NaN,NaN,NaN,NaN,Rural,NaN,NaN,NaN,Not Applicable,Not Applicable
108,"Fast Pace Medical Clinic, Pllc - Sweetwater",883905,Rural Health Clinic,02/01/2022,For Profit Individual,Qcor,Monroe County,TN,"791 New Highway 68, Ste 1",Sweetwater,...,NaN,NaN,NaN,NaN,Rural,NaN,NaN,NaN,Not Applicable,Not Applicable
110,"Fast Pace Medical Clinic, Pllc - Winchester",883906,Rural Health Clinic,02/01/2022,For Profit Individual,Qcor,Franklin County,TN,2401 Decherd Blvd,Winchester,...,NaN,NaN,NaN,NaN,Rural,NaN,NaN,NaN,Not Applicable,Not Applicable


In [ ]:
#rhc.to_csv('../capstone_data/rhc_tn.csv')